In [2]:
import requests
import pandas as pd
import time 
import numpy as np 
#from google.colab import drive
#from google.colab import files

# The Basics of Book Bias
## Analyzing trends of bestselling books on different platforms
Gesna Aggarwal, Brendan DeMilt, Lydia Hu

## New York Times API - Data Extraction and Compilation  

Our team used the New York Times Book API. This API provides information about book reviews and The New York Times Best Sellers lists.

According to the New York Time's [website](https://www.nytimes.com/books/best-sellers/methodology/),  

> "The (bestseller) rankings reflect unit sales reported on a confidential basis by vendors offering a wide range of general interest titles published in the United States. Every week, thousands of diverse selling locations report their actual sales on hundreds of thousands of individual titles. The panel of reporting retailers is comprehensive and reflects sales in tens of thousands of stores of all sizes and demographics across the United States."

In order to get the bestseller data, there were several steps in the process 

1.   Our team had to register an account with the New York Times to get a key to access the data. 
2.  We created a function that returned a list of all the dates in 2018 in "YYYY-MM-DD" format. This is the format the Book API takes in date data. 
3. The New York Times has bestsellers in 61 different categories. In order to narrow down the data that we analyze, we chose the two most general categories from this list. Namely - ***Combined print and e-book fiction*** and  ***Combined print and e-book non-fiction***
4. API calls were made for each day of 2018 for these two categories. All the data returned by the New York Times API is in the JSON format. This data was converted into a pandas dataframe and concatenated into one unified dataframe. 
5. According to the guidelines for using the New York Times API, you can only make one API call every 6 seconds. Thus, creating this dataframe took a total of 73 minutes. 
6. Once the dataframe was created, we were able to store it in the form of a CSV file. This CSV file is what we will be using for the rest of the analysis. 


In [0]:
nyt_key = "QKADCGTZwzGLGNw0ANu79NoNHAvAE5m3"

#returns a list of all dates in 2018 in the YYYY-MM-DD format to pass into the New York Times API 
def get2018Dates(): 
  allDates = [] 
  for i in range(1, 13):
    if i == 1 or i == 3 or i == 5 or i == 7 or i == 8 or i == 10 or i == 12: 
      for j in range(1, 32): 
        allDates.append("2018-" + str('%02d' % i) + "-" + str ('%02d' % j))
    if i == 2: 
      for j in range(1, 29): 
        allDates.append("2018-" + str('%02d' % i) + "-" + str ('%02d' % j))
    if i == 4 or i == 6 or i == 9 or i == 11: 
      for j in range(1, 31): 
        allDates.append("2018-" + str('%02d' % i) + "-" + str ('%02d' % j))
  return allDates

#get the type of books featured in the bestseller category for New York Times 
getTypesOfBooks = requests.get('https://api.nytimes.com/svc/books/v3/lists/names.json?api-key='+nyt_key)
nyt_data = pd.DataFrame.from_dict(getTypesOfBooks.json()['results'])
typesOfBooks = [] 
typesOfBooks.append(nyt_data.list_name_encoded.unique()[0])
typesOfBooks.append(nyt_data.list_name_encoded.unique()[1])

#get all data for all books in 2018 and compile it into one dataset 
all2018Dates = get2018Dates()
finalDataFrame = pd.DataFrame()

for date in all2018Dates: 
  for bookType in typesOfBooks: 
    apiCall = requests.get('https://api.nytimes.com/svc/books/v3/lists/' + 
                           bookType + '/' + date + '.json?api-key=' + nyt_key) 
    time.sleep(6)
    if 'results' in apiCall.json(): 
      test3 = apiCall.json()['results']
      dataFrame = pd.DataFrame(test3['books'])
      dataFrame['date'] = date
      dataFrame['typeOfBook'] = bookType
      finalDataFrame = pd.concat([dataFrame, finalDataFrame])


In [5]:
#Mounting google drive to google collaboratory and exporting the pandas dataframe to google drive as a .csv file 
drive.mount('drive')
finalDataFrame.to_csv('data2.csv')
!cp data2.csv drive/My\ Drive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [6]:
#uploading the csv file to google colab 
uploaded = files.upload()

Saving data2.csv to data2 (1).csv


# New York Times - Data Cleaning 

As mentioned before, we uploaded our code in the form of a .csv file. The original New York Times dataframe had many columns we would not need in our analysis. So before grouping our data by ISBNs, we dropped many columns and kept the following eight: 


1.   Rank_List = A list of all the ranks the book has had on the bestseller list. 
2.   Weeks_On_List = Total number of weeks the book has been on the bestseller list. 
3. ISBN13 = The ISBN13 of the book 
4. Publisher = The publisher of the book 
5. Title = The title of the book 
6. Author = The author of the book 
7. Contributor = The contributor of the book 
8. Dates = The dates the book was on the bestseller list 
9. Book_Type = The type of book it is (combined-print-and-e-book-nonfiction
or combined-print-and-e-book-fiction) 





In [3]:
data = pd.read_csv('data2.csv')

#dropping columns that we will not be using 
data = data.drop(['asterisk', 'dagger', 'amazon_product_url', 'book_review_link', 'first_chapter_link', 'book_image',
                 'book_image_width', 'book_image_height', 'buy_links', 'book_uri', 'article_chapter_link', 'isbns', 
                 'contributor_note', 'sunday_review_link', 'description', 'price', 'primary_isbn10', 'age_group'], axis=1)

#grouping by the ISBN13 
grouped = data.groupby('primary_isbn13')

#creating the final dataframe 
finalDataFrame = pd.DataFrame(columns = ['Rank_List', 'Weeks_On_List', 'ISBN13',
                                         'Publisher', 'Title', 'Author', 
                                         'Contributor', 'Dates', 'Book_Type']) 
iter = 0 
for idx, dataframe in grouped: 
    #create a list of all the days and ranks for the particular book on the bestseller list 
    dates = [] 
    ranks = [] 
    for rows in dataframe.iterrows(): 
        dates.append(rows[1]['date'])
        ranks.append(rows[1]['rank'])
    dates = np.unique(dates)
    ranks = np.unique(ranks)

    #add data to the finalDataFrame 
    finalDataFrame.loc[iter] = [ranks, dataframe['weeks_on_list'].iloc[0], 
                                dataframe['primary_isbn13'].iloc[0], 
                                dataframe['publisher'].iloc[0],
                                dataframe['title'].iloc[0],
                                dataframe['author'].iloc[0], 
                                dataframe['contributor'].iloc[0], dates, 
                                dataframe['typeOfBook'].iloc[0]]
    iter = iter + 1

finalDataFrame.head(10)

,Rank_List,Weeks_On_List,ISBN13,Publisher,Title,Author,Contributor,Dates,Book_Type
0,"[1, 2, 3, 9, 11, 15]",12,9780060899226,Ecco,KITCHEN CONFIDENTIAL,Anthony Bourdain,by Anthony Bourdain,"[2018-06-25, 2018-06-26, 2018-06-27, 2018-06-2...",combined-print-and-e-book-nonfiction
1,[6],1,9780061998065,HarperCollins,MEDIUM RAW,Anthony Bourdain,by Anthony Bourdain,"[2018-07-09, 2018-07-10, 2018-07-11, 2018-07-1...",combined-print-and-e-book-nonfiction
2,"[3, 4, 5, 8, 11, 13, 14]",8,9780062294449,Harper,THE NEXT PERSON YOU MEET IN HEAVEN,Mitch Albom,by Mitch Albom,"[2018-10-22, 2018-10-23, 2018-10-24, 2018-10-2...",combined-print-and-e-book-fiction
3,"[7, 9, 11, 13, 15]",73,9780062300546,HarperCollins,HILLBILLY ELEGY,JD Vance,by J.D. Vance,"[2018-01-08, 2018-01-09, 2018-01-10, 2018-01-1...",combined-print-and-e-book-nonfiction
4,"[3, 6, 11, 12, 13, 14, 15]",85,9780062300553,HarperCollins,HILLBILLY ELEGY,JD Vance,by J.D. Vance,"[2018-05-14, 2018-05-15, 2018-05-16, 2018-05-1...",combined-print-and-e-book-nonfiction
5,"[13, 14, 15]",21,9780062316097,Harper,SAPIENS,Yuval Noah Harari,by Yuval Noah Harari,"[2018-01-22, 2018-01-23, 2018-01-24, 2018-01-2...",combined-print-and-e-book-nonfiction
6,"[1, 14]",38,9780062316103,Harper,SAPIENS,Yuval Noah Harari,by Yuval Noah Harari,"[2018-03-19, 2018-03-20, 2018-03-21, 2018-03-2...",combined-print-and-e-book-nonfiction
7,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 15]",52,9780062316110,Harper,SAPIENS,Yuval Noah Harari,by Yuval Noah Harari,"[2018-05-28, 2018-05-29, 2018-05-30, 2018-05-3...",combined-print-and-e-book-nonfiction
8,"[1, 2, 4, 5, 7, 9, 12, 14, 15]",15,9780062319784,Harper,I'LL BE GONE IN THE DARK,Michelle McNamara,by Michelle McNamara,"[2018-03-12, 2018-03-13, 2018-03-14, 2018-03-1...",combined-print-and-e-book-nonfiction
9,[2],9,9780062319807,Harper,I'LL BE GONE IN THE DARK,Michelle McNamara,by Michelle McNamara,"[2018-05-07, 2018-05-08, 2018-05-09, 2018-05-1...",combined-print-and-e-book-nonfiction


# Goodreads API
Goodreads is a social website for readers owned by Amazon, allowing them to track, organize, rate, and interact with friends and authors to collaborate in groups and lists to discuss books. So addition to expected data about books (ISBN, publication date), Goodreads contains data about how readers of a book interact with each other. Goodreads is different from Amazon, NYT, and publishers' sites because it crowdsources some of its information. For instance, the genre of a book depends on how users personally categorize the book in their shelves. Goodreads also allows readers to add metadata about a book (some subjective), for instance, its tone, pace, literary devices used. 

Goodreads uses its own book IDs in addition to having ISBNs. Some of the categorization ambiguities with this system are that different editions of a book will have different ISBNs, like a collector's edition, a movie-tie edition, a signed edition, etc. Goodreads would then have a separate entry for each of these editions and users can go in and separately review each of these editions. This makes it so there's not really one immediately available source of truth for how Goodreads users feel about a book because of how many editions there might be reviewed. 

In [4]:
gr_key = 'LR4gZ9vJHllxG6Xfp8eKnQ'

best_of_18 = ['0062498533,1501173219,1619635194,0425232204,1250080401,0553418025,0307588378,0062459368,0545586178']

params = (
    ('key', gr_key),
    ('isbns', best_of_18),
    ('format', 'json'),
)
review_request = 'https://www.goodreads.com/book/review_counts.json'
goodreads_result = requests.get(review_request, params=params)

gr_data = pd.DataFrame.from_dict(goodreads_result.json()["books"])
gr_data.head(10)


,id,isbn,isbn13,ratings_count,reviews_count,text_reviews_count,work_ratings_count,work_reviews_count,work_text_reviews_count,average_rating
0,32075671,0062498533,9780062498533,294679,67012,37470,370499,68982,48460,4.52
1,34006942,1501173219,9781501173219,3141,8672,432,866930,1834856,69565,4.33
2,31451175,1619635194,9781619635197,1022,1634,159,251754,435130,33871,4.66
3,6588662,0425232204,9780425232200,9577,16503,1210,1946514,2716640,82604,4.47
4,25587463,1250080401,9781250080400,5447,3193,837,565393,948519,56011,4.57
5,44334843,0307588378,9780307588371,322,627,15,2073437,3090584,129221,4.06
6,27161228,0062459368,9780062459367,969,1502,116,438385,857627,44884,4.00
7,16280929,0545586178,9780545586177,2378,176,123,2260669,2720527,92606,4.29


# OpenLibrary API
Since both of these datasets do not contain all of the data we need, we need to find another source of information to fill out details on these books. One such resource we are utilizing is Open Library, which is an open source library....library whose mission is to catalog the world's books, which is super useful for our needs. Therefore, we wrote a few helper methods using the Open Library API to help add standardized information to the data we already have for both tables

In [5]:
OL_BASE = "https://openlibrary.org/api/books"

def fetch_book(isbn):
    test = requests.get(OL_BASE+"?bibkeys=ISBN:"+str(isbn)+"&jscmd=data&format=json")
    if test.status_code == 200:
        return test.json()["ISBN:"+str(isbn)]
    else:
        return None

def fetch_batch(arr):
    query = ""
    for i in arr:
        query += "ISBN:"+str(i)+","
    query = query[0:-1]
    test = requests.get(OL_BASE+"?bibkeys="+str(query)+"&jscmd=data&format=json")
    if test.status_code == 200:
        resp = test.json()
        ret = [None]*len(arr)
        for i in range(len(arr)):
            try:
                ret[i] = resp["ISBN:"+str(arr[i])]
            except KeyError:
                continue
        return ret
    else:
        return None

def fetch_batch_subsection(arr, subtitle):
    book = fetch_batch(arr)
    ret = [None]*len(book)
    for i in range(len(book)):
        try:
            sections = book[i][subtitle]
            ret[i] = [j["name"] for j in sections]
        except:
            continue
    return ret

def fetch_batch_section(arr, title):
    books = fetch_batch(isbn_arr)
    ret = [None]*len(books)
    for i in range(len(books)):
        try:
            ret[i] = books[i][title]
        except:
            continue
    return ret
    
def get_genres(isbn_arr):
    return fetch_batch_subsection(isbn_arr,"subjects")

def get_titles(isbn_arr):
    return fetch_batch_section(isbn_arr, "title")

def get_publishers(isbn_arr):
    return fetch_batch_subsection(isbn_arr,"publishers")

def get_publisher_locations(isbn_arr):
    return fetch_batch_subsection(isbn_arr,"publish_places")
     
def get_page_counts(isbn_arr):
    return fetch_batch_section(isbn_arr, "number_of_pages")

def get_publish_date(isbn_arr):
    return fetch_batch_section(isbn_arr, "publish_date")

The Goodreads API call provides a bunch of user generated data for it's top rated books. However a lot of this data we don't need, so we will be dropping all columns except for: 

*   **isbn:** to connect to Open Library
*   **work_ratings_count**: total number of user star ratings without text review
*   **work_text_reviews_count:** total number of user reviews with text
*   **average_rating:** aggregate user rating of the book out of 5


In [6]:
gr_data = gr_data[["isbn", "work_ratings_count", "work_text_reviews_count", "average_rating"]].copy()

Now, we use the OpenLibrary API functions to add standard information goodreads does not provide us, as well as common information formats which will make comparing the Goodreads and New York Times datasets more easily. Here we are adding the title of the book, associated genres if any, it's publishers, the publication date, the publisher location, and the page count of the book.

In [7]:
isbn_arr = list(gr_data["isbn"])
gr_data["book_title"] = pd.Series(get_titles(isbn_arr))
gr_data["genres"] = pd.Series(get_genres(isbn_arr))
gr_data["publishers"] = pd.Series(get_publishers(isbn_arr)).map(lambda x: x[0] if x != None else None)
gr_data["publish_date"] = pd.Series(get_publish_date(isbn_arr)).astype("datetime64")
gr_data["publisher_location"] = pd.Series(get_publisher_locations(isbn_arr)).map(lambda x: x[0] if x != None else None)
gr_data["page_count"] = pd.Series(get_page_counts(isbn_arr))
gr_data.head(10)

,isbn,work_ratings_count,work_text_reviews_count,average_rating,book_title,genres,publishers,publish_date,publisher_location,page_count
0,0062498533,370499,48460,4.52,The Hate U Give,"[Fiction, National Book Awards Longlist, Race ...","Balzer + Bray, an imprint of HarperCollinsPubl...",2017-01-01,"New York, NY, USA",444.0
1,1501173219,866930,69565,4.33,All the Light We Cannot See: A Novel,None,Scribner,2017-04-04,None,544.0
2,1619635194,251754,33871,4.66,None,None,None,NaT,None,NaN
3,0425232204,1946514,82604,4.47,The Help,"[novels, Fiction, Civil rights movements, Hous...",Berkley Books,2011-04-01,"New York, USA",NaN
4,1250080401,565393,56011,4.57,The Nightingale: A Novel,None,St. Martin's Griffin,2017-04-25,None,608.0
5,0307588378,2073437,129221,4.06,Gone Girl,"[mystery fiction, suspense fiction, Detective ...",Broadway Books,2014-01-01,"New York, USA",422.0
6,0062459368,438385,44884,4.00,None,None,None,NaT,None,NaN
7,0545586178,2260669,92606,4.29,Catching Fire,"[Action/Adventure, fantasy, teen fiction, juve...",Scholastic,2009-01-01,None,NaN


We will now add the same columns to the New York Times dataset that it doesn't already have. This way, when it comes time to compare the two, we have standard data to work with. The only difference with this process is that, instead of using isbn10 to gather data, we will be calling OpenLibrary with the isbn13 number, as fewer books have an isbn10 number in the New York Times dataset. The Open Library API limits each call to have about 200 isbns each, so we're going to have to make 3 calls to get all the data for each column

In [8]:
nyt = finalDataFrame

def createHugeColumn(x, arr):
    split = len(arr)//3
    column = pd.Series(x(arr[:split]))
    column = column.append(pd.Series(x(arr[split:2*split])))
    column = column.append(pd.Series(x(arr[2*split:])))
    column.index = [i for i in range(len(column))]
    return column

isbn13_arr = list(nyt["ISBN13"])
nyt["genres"] = createHugeColumn(get_genres, isbn13_arr)
nyt["publisher_location"] = createHugeColumn(get_publisher_locations, isbn13_arr).map(lambda x: x[0] if x != None else None)
nyt["page_count"] = createHugeColumn(get_page_counts,isbn13_arr)
nyt["OL_title"] = createHugeColumn(get_titles,isbn13_arr)
nyt.head(10)

,Rank_List,Weeks_On_List,ISBN13,Publisher,Title,Author,Contributor,Dates,Book_Type,genres,publisher_location,page_count,OL_title
0,"[1, 2, 3, 9, 11, 15]",12,9780060899226,Ecco,KITCHEN CONFIDENTIAL,Anthony Bourdain,by Anthony Bourdain,"[2018-06-25, 2018-06-26, 2018-06-27, 2018-06-2...",combined-print-and-e-book-nonfiction,"[Cooks, Accessible book, In library, Cocineros...",New York,444.0,The Hate U Give
1,[6],1,9780061998065,HarperCollins,MEDIUM RAW,Anthony Bourdain,by Anthony Bourdain,"[2018-07-09, 2018-07-10, 2018-07-11, 2018-07-1...",combined-print-and-e-book-nonfiction,"[nyt:paperback_nonfiction=2011-04-30, Nonficti...",New York,544.0,All the Light We Cannot See: A Novel
2,"[3, 4, 5, 8, 11, 13, 14]",8,9780062294449,Harper,THE NEXT PERSON YOU MEET IN HEAVEN,Mitch Albom,by Mitch Albom,"[2018-10-22, 2018-10-23, 2018-10-24, 2018-10-2...",combined-print-and-e-book-fiction,"[Amusement rides, Heaven, Older men, Death, Fu...",None,NaN,None
3,"[7, 9, 11, 13, 15]",73,9780062300546,HarperCollins,HILLBILLY ELEGY,JD Vance,by J.D. Vance,"[2018-01-08, 2018-01-09, 2018-01-10, 2018-01-1...",combined-print-and-e-book-nonfiction,None,None,NaN,The Help
4,"[3, 6, 11, 12, 13, 14, 15]",85,9780062300553,HarperCollins,HILLBILLY ELEGY,JD Vance,by J.D. Vance,"[2018-05-14, 2018-05-15, 2018-05-16, 2018-05-1...",combined-print-and-e-book-nonfiction,"[Mountain people, Family, Social mobility, Wor...",None,608.0,The Nightingale: A Novel
5,"[13, 14, 15]",21,9780062316097,Harper,SAPIENS,Yuval Noah Harari,by Yuval Noah Harari,"[2018-01-22, 2018-01-23, 2018-01-24, 2018-01-2...",combined-print-and-e-book-nonfiction,None,None,422.0,Gone Girl
6,"[1, 14]",38,9780062316103,Harper,SAPIENS,Yuval Noah Harari,by Yuval Noah Harari,"[2018-03-19, 2018-03-20, 2018-03-21, 2018-03-2...",combined-print-and-e-book-nonfiction,None,None,NaN,None
7,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 15]",52,9780062316110,Harper,SAPIENS,Yuval Noah Harari,by Yuval Noah Harari,"[2018-05-28, 2018-05-29, 2018-05-30, 2018-05-3...",combined-print-and-e-book-nonfiction,None,None,NaN,Catching Fire
8,"[1, 2, 4, 5, 7, 9, 12, 14, 15]",15,9780062319784,Harper,I'LL BE GONE IN THE DARK,Michelle McNamara,by Michelle McNamara,"[2018-03-12, 2018-03-13, 2018-03-14, 2018-03-1...",combined-print-and-e-book-nonfiction,"[Serial murders, Case studies, Serial murder i...",USA,444.0,The Hate U Give
9,[2],9,9780062319807,Harper,I'LL BE GONE IN THE DARK,Michelle McNamara,by Michelle McNamara,"[2018-05-07, 2018-05-08, 2018-05-09, 2018-05-1...",combined-print-and-e-book-nonfiction,None,None,544.0,All the Light We Cannot See: A Novel


In addition, we'd like to make the book type column something a little more friendly to work with in our analysis, so we will change the column to be 1 if the book is fiction, and 0 if it is nonfiction

In [9]:
nyt["Book_Type"] = nyt["Book_Type"].map(lambda x: 1 if x == "combined-print-and-e-book-fiction" else 0)
nyt.head(10)

,Rank_List,Weeks_On_List,ISBN13,Publisher,Title,Author,Contributor,Dates,Book_Type,genres,publisher_location,page_count,OL_title
0,"[1, 2, 3, 9, 11, 15]",12,9780060899226,Ecco,KITCHEN CONFIDENTIAL,Anthony Bourdain,by Anthony Bourdain,"[2018-06-25, 2018-06-26, 2018-06-27, 2018-06-2...",0,"[Cooks, Accessible book, In library, Cocineros...",New York,444.0,The Hate U Give
1,[6],1,9780061998065,HarperCollins,MEDIUM RAW,Anthony Bourdain,by Anthony Bourdain,"[2018-07-09, 2018-07-10, 2018-07-11, 2018-07-1...",0,"[nyt:paperback_nonfiction=2011-04-30, Nonficti...",New York,544.0,All the Light We Cannot See: A Novel
2,"[3, 4, 5, 8, 11, 13, 14]",8,9780062294449,Harper,THE NEXT PERSON YOU MEET IN HEAVEN,Mitch Albom,by Mitch Albom,"[2018-10-22, 2018-10-23, 2018-10-24, 2018-10-2...",1,"[Amusement rides, Heaven, Older men, Death, Fu...",None,NaN,None
3,"[7, 9, 11, 13, 15]",73,9780062300546,HarperCollins,HILLBILLY ELEGY,JD Vance,by J.D. Vance,"[2018-01-08, 2018-01-09, 2018-01-10, 2018-01-1...",0,None,None,NaN,The Help
4,"[3, 6, 11, 12, 13, 14, 15]",85,9780062300553,HarperCollins,HILLBILLY ELEGY,JD Vance,by J.D. Vance,"[2018-05-14, 2018-05-15, 2018-05-16, 2018-05-1...",0,"[Mountain people, Family, Social mobility, Wor...",None,608.0,The Nightingale: A Novel
5,"[13, 14, 15]",21,9780062316097,Harper,SAPIENS,Yuval Noah Harari,by Yuval Noah Harari,"[2018-01-22, 2018-01-23, 2018-01-24, 2018-01-2...",0,None,None,422.0,Gone Girl
6,"[1, 14]",38,9780062316103,Harper,SAPIENS,Yuval Noah Harari,by Yuval Noah Harari,"[2018-03-19, 2018-03-20, 2018-03-21, 2018-03-2...",0,None,None,NaN,None
7,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 15]",52,9780062316110,Harper,SAPIENS,Yuval Noah Harari,by Yuval Noah Harari,"[2018-05-28, 2018-05-29, 2018-05-30, 2018-05-3...",0,None,None,NaN,Catching Fire
8,"[1, 2, 4, 5, 7, 9, 12, 14, 15]",15,9780062319784,Harper,I'LL BE GONE IN THE DARK,Michelle McNamara,by Michelle McNamara,"[2018-03-12, 2018-03-13, 2018-03-14, 2018-03-1...",0,"[Serial murders, Case studies, Serial murder i...",USA,444.0,The Hate U Give
9,[2],9,9780062319807,Harper,I'LL BE GONE IN THE DARK,Michelle McNamara,by Michelle McNamara,"[2018-05-07, 2018-05-08, 2018-05-09, 2018-05-1...",0,None,None,544.0,All the Light We Cannot See: A Novel
